In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
train_df = pd.read_pickle("/content/drive/My Drive/CMPE 257/CMPE 257 Colab/257 Sensitive Data Input Guardrail/train_df_embedding.pkl")
test_df = pd.read_pickle("/content/drive/My Drive/CMPE 257/CMPE 257 Colab/257 Sensitive Data Input Guardrail/test_df_embedding.pkl")


Mounted at /content/drive


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# vectorize text with TF-IDF
vectorizer = TfidfVectorizer(max_features = 1500, ngram_range = (1, 2), stop_words = "english")

# svm, make sure to adjust c
svm_pipeline = Pipeline([("tfidf", vectorizer),("svm", LinearSVC(C = 10))])

# train model
svm_pipeline.fit(train_df["source_text"], train_df["is_sensitive"])

# evaluate
y_pred = svm_pipeline.predict(test_df["source_text"])

print("Accuracy:", accuracy_score(test_df["is_sensitive"], y_pred))
print(classification_report(test_df["is_sensitive"], y_pred, target_names = ["Not Sensitive", "Sensitive"]))


Accuracy: 0.916058394160584
               precision    recall  f1-score   support

Not Sensitive       0.64      0.60      0.62       908
    Sensitive       0.95      0.96      0.95      7038

     accuracy                           0.92      7946
    macro avg       0.79      0.78      0.79      7946
 weighted avg       0.91      0.92      0.91      7946



In [5]:
# Embeddings and PCA with 80% variance

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# building feature matrices from embedding column
X_train = np.vstack(train_df["embedding"].values)  # shape - (n_train, embedding_dim)
X_test  = np.vstack(test_df["embedding"].values)   # shape - (n_test, embedding_dim)

y_train = train_df["is_sensitive"].values
y_test  = test_df["is_sensitive"].values


# scaling embeddings before logistic regression
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


# fit pca with 114 components
pca = PCA(n_components = 114) # 114 components to capture 80% of variance
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# svm, make sure to adjust c
svm_pipeline = Pipeline([("pca", pca),("svm", LinearSVC(C = 10))])

# train model
svm_pipeline.fit(X_train, y_train)

# evaluate
y_pred = svm_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(test_df["is_sensitive"], y_pred))
print(classification_report(test_df["is_sensitive"], y_pred, target_names = ["Not Sensitive", "Sensitive"]))

Accuracy: 0.929146740498364
               precision    recall  f1-score   support

Not Sensitive       0.74      0.59      0.66       908
    Sensitive       0.95      0.97      0.96      7038

     accuracy                           0.93      7946
    macro avg       0.84      0.78      0.81      7946
 weighted avg       0.92      0.93      0.93      7946

